In [0]:
from pyspark.sql.types import StructType, StructField, DateType,IntegerType, StringType, DataType

In [0]:
races_schema = StructType(fields=[StructField("raceId", IntegerType(), False ),
                                    StructField("year", IntegerType(), True ),
                                    StructField("round", IntegerType(), True ),
                                    StructField("circuitId", IntegerType(), True),
                                    StructField("name", StringType(), True ),
                                    StructField("date", DateType(), True ),
                                    StructField("time", StringType(), True ),
                                    StructField("url", StringType(), True ),
                                   ])

In [0]:
races_df = spark.read \
.option("header", True) \
.schema(races_schema) \
.csv("/FileStore/tables/races.csv")

In [0]:
from pyspark.sql.functions import current_timestamp, to_timestamp, concat, col, lit

In [0]:
race_with_timestamp_df = races_df.withColumn("ingest_date", current_timestamp()) \
.withColumn("race_timestamp", to_timestamp(concat(col('date'),lit(''),col('time')),'yyyy-mm-dd HH:mm:ss'))

In [0]:
race_selected_df = race_with_timestamp_df.select(col('raceid').alias('race_id'),col('year').alias('race_year'),col('round'),col('circuitId').alias('circuit_Id'),col('name'),col('ingest_date'),col('race_timestamp'))

In [0]:
race_selected_df.write.mode('overwrite').partitionBy('race_year').parquet('/mnt/formula1dl/processed/races')